With code like the one in this notebook, we have tried many libraries to extract information from PDF files without success.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install langchain sentence-transformers pypdf qdrant-client pdfminer pip install pdfminer.six pymupdf unstructured pdf2image unstructured_inference pikepdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 29.6 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.1.0 which is incompatible.


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant
# from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import DirectoryLoader, PyPDFLoader, PDFMinerLoader, PyMuPDFLoader, UnstructuredPDFLoader, MathpixPDFLoader, PDFPlumberLoader
from langchain.document_loaders import UnstructuredHTMLLoader, BSHTMLLoader

In [ ]:
# model_name = "BAAI/bge-large-en"
model_name = "all-mpnet-base-v2"
# model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    # model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
%cd ..

/content/drive/MyDrive/Anyone_AI/Final_Project


In [ ]:
%cd drive/MyDrive/Anyone_AI/Final_Project/Raw_Data

/content/drive/MyDrive/Anyone_AI/Final_Project/Raw_Data


In [ ]:
folder = 'zoom-video-communications-inc'

In [ ]:
loader = DirectoryLoader(
    folder,
    glob="*.pdf",
    show_progress=True,
    loader_cls=UnstructuredPDFLoader
)

documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

texts = text_splitter.split_documents(documents)

  0%|          | 0/2 [00:00<?, ?it/s][nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
100%|██████████| 2/2 [00:50<00:00, 25.43s/it]


In [ ]:
len(texts)/2

3983.0

In [ ]:
loader = UnstructuredHTMLLoader("zm-20230131.html")
data = loader.load()

In [ ]:
len(data[0].page_content)

435616

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

texts = text_splitter.split_documents(data)

In [ ]:
len(texts)

632

In [ ]:
texts[316]

Document(page_content='rumors and market speculation involving us or other companies in our industry;\n\nactual or anticipated changes in our results of operations or fluctuations in our results of operations;\n\nactual or anticipated developments in our business, our competitors’ businesses, or the competitive landscape generally;\n\nlitigation involving us, our industry, or both, or investigations by regulators into our operations or those of our competitors;\n\ndevelopments or disputes concerning our intellectual property or other proprietary rights;\n\nannounced or completed acquisitions of businesses, products, services, or technologies by us or our competitors;\n\nnew laws or regulations or new interpretations of existing laws or regulations applicable to our business;\n\nchanges in accounting standards, policies, guidelines, interpretations, or principles;\n\nany significant change in our management; and\n\ngeneral economic conditions and slow or negative growth of our markets.'

In [ ]:
loader = BSHTMLLoader("zm-20230131.html")
data = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

texts = text_splitter.split_documents(data)

In [ ]:
%ls

'doc preguntas y respuestas.docx'                  Raw_Data/
 Download_files_from_S3.ipynb                      Untitled1.ipynb
 Extract_info_from_pdf.ipynb                       zm-20230131.html
 pdf_processing_with_langchain_PyPDFLoader.ipynb   zoom_video_vectors/


In [ ]:
db = Qdrant.from_documents(
    texts,
    embeddings,
    path="html_zoom_video_vectors",
    collection_name="vector_db"
)

# CID Thing

In [ ]:
import re

def cidToChar(cidx):
    return chr(int(re.findall(r'\(cid\:(\d+)\)',cidx)[0]) + 29)

In [ ]:
def cids_to_text(xx):
  for x in xx.split('\n'):
    if x != '' and x != '(cid:3)':         # merely to compact the output
      abc = re.findall(r'\(cid\:\d+\)',x)
      if len(abc) > 0:
          for cid in abc: x=x.replace(cid, cidToChar(cid))
      print(repr(x).strip("'"))

In [ ]:
texts[2000].page_content

'(cid:75)(cid:67)(cid:73)(cid:64)(cid:56) (cid:66)(cid:67)(cid:72) (cid:60)(cid:53)(cid:74)(cid:57) (cid:67)(cid:55)(cid:55)(cid:73)(cid:70)(cid:70)(cid:57)(cid:56) (cid:53)(cid:54)(cid:71)(cid:57)(cid:66)(cid:72) (cid:72)(cid:60)(cid:57) (cid:55)(cid:73)(cid:71)(cid:72)(cid:67)(cid:65)(cid:57)(cid:70) (cid:55)(cid:67)(cid:66)(cid:72)(cid:70)(cid:53)(cid:55)(cid:72)(cid:11)'

In [ ]:
cids_to_text('(cid:3)(cid:3)(cid:70)')

  c


In [ ]:
cids_to_text(texts[2000].page_content)

h`f]U _`e YRgV `TTfccVU RSdV_e eYV Tfde`^Vc T`_ecRTe(


In [ ]:
xx = '''
(cid:3)
(cid:3)
(cid:3)
(cid:3)

(cid:3)
(cid:3)
(cid:3)

Nr. 3 | 2021

März 2021

(cid:3)
(cid:57)(cid:72)(cid:85)(cid:75)(cid:68)(cid:81)(cid:71)(cid:79)(cid:88)(cid:81)(cid:74)(cid:72)(cid:81)(cid:3)(cid:71)(cid:72)(cid:86)(cid:3)(cid:42)(cid:72)(cid:80)(cid:72)(cid:76)(cid:81)(cid:71)(cid:72)(cid:85)(cid:68)(cid:87)(cid:72)(cid:86)(cid:3)
(cid:3)
(cid:54)(cid:70)(cid:75)(cid:88)(cid:79)(cid:72)(cid:81)(cid:3)
(cid:3)
(cid:54)(cid:82)(cid:93)(cid:76)(cid:68)(cid:79)(cid:72)(cid:3)(cid:39)(cid:76)(cid:72)(cid:81)(cid:86)(cid:87)(cid:72)(cid:3)
(cid:3)
(cid:48)(cid:76)(cid:87)(cid:87)(cid:72)(cid:76)(cid:79)(cid:88)(cid:81)(cid:74)(cid:72)(cid:81)(cid:3)(cid:39)(cid:82)(cid:85)(cid:73)(cid:89)(cid:72)(cid:85)(cid:72)(cid:76)(cid:81)(cid:72)(cid:3)
(cid:3)
(cid:48)(cid:76)(cid:87)(cid:87)(cid:72)(cid:76)(cid:79)(cid:88)(cid:81)(cid:74)(cid:72)(cid:81)(cid:3)(cid:68)(cid:88)(cid:86)(cid:90)(cid:108)(cid:85)(cid:87)(cid:76)(cid:74)(cid:72)(cid:85)(cid:3)(cid:57)(cid:72)(cid:85)(cid:72)(cid:76)(cid:81)(cid:72)(cid:3)
(cid:3)
(cid:48)(cid:76)(cid:87)(cid:87)(cid:72)(cid:76)(cid:79)(cid:88)(cid:81)(cid:74)(cid:72)(cid:81)(cid:3)(cid:46)(cid:76)(cid:85)(cid:70)(cid:75)(cid:74)(cid:72)(cid:80)(cid:72)(cid:76)(cid:81)(cid:71)(cid:72)(cid:81)(cid:3)

(cid:20)(cid:3)

(cid:23)(cid:3)

(cid:20)(cid:21)(cid:3)

(cid:21)(cid:20)(cid:3)

(cid:21)(cid:24)(cid:3)

Mitteilungsblatt Neerach | Gemeindeverwaltung Neerach | Binzmühlestrasse 14 | 8173 Neerach
044 859 16 16 | einwohnerkontrolle@neerach.ch | www.neerach.ch

(cid:3)(cid:3)(cid:3)(cid:3)(cid:3)(cid:3)(cid:3)(cid:3)
(cid:3)
(cid:3)
(cid:3)
'''


In [ ]:
cids_to_text(xx)

Nr. 3 | 2021
März 2021
Verhandlungen des Gemeinderates 
Schulen 
Soziale Dienste 
Mitteilungen Dorfvereine 
Mitteilungen ausw\x89rtiger Vereine 
Mitteilungen Kirchgemeinden 
1 
4 
12 
21 
25 
Mitteilungsblatt Neerach | Gemeindeverwaltung Neerach | Binzmühlestrasse 14 | 8173 Neerach
044 859 16 16 | einwohnerkontrolle@neerach.ch | www.neerach.ch
        


In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
def get_links(query):
    """Obtiene los enlaces HTML de la página web de búsqueda de la SEC.

    Args:
        query: La cadena de búsqueda.

    Returns:
        Una lista de enlaces HTML.
    """

    # Realiza una solicitud HTTP a la página web de búsqueda.
    response = requests.get("https://www.sec.gov/edgar/search/" + query)

    # Si la solicitud fue exitosa, parsea el HTML de la respuesta.
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")

        # Obtiene una lista de todos los elementos <a> de la página web.'13209735d94afdfd58776a138929df63'
        links = soup.find_all("a")

        # Filtra los enlaces que contienen el texto "Document".
        links = [link for link in links if "Document" in link.text]

        # Devuelve la lista de enlaces.
        return links

    # Si la solicitud no fue exitosa, devuelve una lista vacía.
    else:
        return []

In [ ]:
# Ejemplo de uso.
links = get_links("#/"+
                  "category=form-cat1"+
                  "&ciks=0000718877"+
                  "&entityName=Activision%2520Blizzard%252C%2520Inc.%2520(CIK%25200000718877)")

# Imprime la lista de enlaces.
for link in links:
    print(link["href"])

In [ ]:
links

[]

In [ ]:
response = requests.get("https://www.sec.gov/edgar/search/" +
                        "#/"+
                        "category=form-cat1"+
                        "&ciks=0000718877"+
                        "&entityName=Activision%2520Blizzard%252C%2520Inc.%2520(CIK%25200000718877)")

In [ ]:
response.text

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />\n<title>SEC.gov | Request Rate Threshold Exceeded</title>\n<style>\nhtml {height: 100%}\nbody {height: 100%; margin:0; padding:0;}\n#header {background-color:#003968; color:#fff; padding:15px 20px 10px 20px;font-family:Arial, Helvetica, sans-serif; font-size:20px; border-bottom:solid 5px #000;}\n#footer {background-color:#003968; color:#fff; padding:15px 20px;font-family:Arial, Helvetica, sans-serif; font-size:20px;}\n#content {max-width:650px;margin:60px auto; padding:0 20px 100px 20px; background-image:url(seal_bw.png);background-repeat:no-repeat;background-position:50% 100%;}\nh1 {font-family:Georgia, Times, serif; font-size:20px;}\nh2 {text-align:center; font-family:Georgia, Times, serif; font-size:20px; width:100%; border-bottom:solid #99

In [ ]:
soup = BeautifulSoup(response.text, "html.parser")

In [ ]:
soup

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>SEC.gov | Request Rate Threshold Exceeded</title>
<style>
html {height: 100%}
body {height: 100%; margin:0; padding:0;}
#header {background-color:#003968; color:#fff; padding:15px 20px 10px 20px;font-family:Arial, Helvetica, sans-serif; font-size:20px; border-bottom:solid 5px #000;}
#footer {background-color:#003968; color:#fff; padding:15px 20px;font-family:Arial, Helvetica, sans-serif; font-size:20px;}
#content {max-width:650px;margin:60px auto; padding:0 20px 100px 20px; background-image:url(seal_bw.png);background-repeat:no-repeat;background-position:50% 100%;}
h1 {font-family:Georgia, Times, serif; font-size:20px;}
h2 {text-align:center; font-family:Georgia, Times, serif; font-size:20px; width:100%; border-bottom:solid #999 1px;padding

In [ ]:
# Tesseract OCR